In [1]:
import monte_carlo.montecarlo as mt
import matplotlib.pyplot as plt
import input_montecarlo as imt

In [7]:
imt.captura_arquivo()

Escolha o arquivo para upload
As datas não estão no formato correto (YYYY-mm). Exemplo: 2020-04
